##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ejemplo de poda en Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/model_optimization/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

Le presentamos un ejemplo completo de *poda de pesos* basada en magnitudes.

### Otras paginas

Para obtener una introducción a qué es la poda de pesos y determinar si debe usarla (incluido lo que se admite), consulte la [página de descripción general](https://www.tensorflow.org/model_optimization/guide/pruning).

Para encontrar rápidamente las API que necesita para su caso de uso (más allá de podar completamente un modelo con dispersión de 80 %), consulte la [guía completa](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md).

### Resumen

En este tutorial, podrás:

1. Entrenar un modelo `tf.keras` para MNIST desde cero.
2. Ajustar el modelo aplicando la API de poda y comprobar la precisión.
3. Crear modelos de TF y TFLite 3 veces más pequeños a partir de la poda.
4. Crear un modelo de TFLite 10 veces más pequeño combinando la poda y la cuantización posentrenamiento.
5. Ver la persistencia de la precisión desde TF a TFLite.

## Preparación

In [ ]:
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

## Entrenar un modelo para MNIST sin poda

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Evalúe la precisión de la prueba de referencia y guarde el modelo para su uso posterior.

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

## Ajustar el modelo preentrenado con poda


### Definir el modelo

Aplicará la poda a todo el modelo y lo verá en el resumen del modelo.

En este ejemplo, el modelo empieza con un 50 % de dispersión (50 % de ceros en los pesos) y finaliza con un 80 % de dispersión.

En la [guía completa](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md), puede ver cómo podar algunas capas para mejorar la precisión del modelo.

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

### Entrenar y evalar el modelo con respecto al punto de referencia

Ajustar con poda durante dos épocas.

Se requiere `tfmot.sparsity.keras.UpdatePruningStep` durante el entrenamiento, y `tfmot.sparsity.keras.PruningSummaries` proporciona registros para rastrear el progreso y la depuración.

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Para este ejemplo, hay una pérdida mínima en la precisión de la prueba después de la poda, en comparación con el punto de referencia.

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Los registros muestran el progreso de la dispersión por capa.

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

Los usuarios que no son de Colab pueden ver [los resultados de una ejecución anterior](https://tensorboard.dev/experiment/sRQnrycaTMWQOaswXzClYA/#scalars&_smoothingWeight=0) de este bloque de código en [TensorBoard.dev](https://tensorboard.dev/).

## Crear modelos 3 veces más pequeños a partir de la poda

Tanto `tfmot.sparsity.keras.strip_pruning` como la aplicación de un algoritmo de compresión estándar (por ejemplo, mediante gzip) son necesarios para ver los beneficios de la compresión de la poda.

- `strip_pruning` es necesario ya que elimina cada tf.Variable que la poda solo necesita durante el entrenamiento, que de otro modo aumentaría el tamaño del modelo durante la inferencia.
- Es necesario aplicar un algoritmo de compresión estándar ya que las matrices de peso serializadas tienen el mismo tamaño que tenían antes de la poda. Sin embargo, la poda hace que la mayoría de los pesos sean ceros, lo que supone una redundancia adicional que los algoritmos pueden usar para comprimir aún más el modelo.

Primero, cree un modelo comprimible para TensorFlow.

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Luego, cree un modelo comprimible para TFLite.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Defina una función ayudante para comprimir los modelos mediante gzip y medir el tamaño comprimido.

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

Compare y vea que los modelos son 3 veces más pequeños debido a la poda.

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

## Cree un modelo 10 veces más pequeño combinando poda y cuantización

Puede aplicar la cuantización posentrenamiento al modelo podado para obtener beneficios adicionales.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

## Ver la persistencia de la precisión desde TF a TFLite

Defina una función ayudante para evaluar el modelo de TFLite en el conjunto de datos de prueba.

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

Evalúe el modelo podado y cuantizado y verá que la precisión de TensorFlow persiste en el backend de TFLite.

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

## Conclusión

En este tutorial, aprendió cómo crear modelos dispersos con la API del kit de herramientas de optimización de modelos de TensorFlow para TensorFlow y TFLite. Luego combinó la poda con la cuantización posentrenamiento para obtener beneficios adicionales.

Creó un modelo 10 veces más pequeño para MNIST, con una diferencia de precisión mínima.

Le recomendamos que pruebe esta nueva capacidad, que puede ser particularmente importante para la implementación en entornos con recursos limitados.
